# 简单持续集成方案

在前面的案例当中，我们学习了如何将应用打包，再部署到Kubernetes集群当中。但是从实际使用方法来看，不可能让每个工程师自己手动完成打包和部署。为了简化工程师的操作，标准化整个部署流程，我们需要引入持续集成解决方案来管理整个流程，让代码顺利的从工程师的电脑上部署到集群当中。这里我们将使用一些简单的开源软件来搭建一套持续集成系统，完成Kubernetes集群当中业务系统的自动更新。

## 部署Gogs

[Gogs](https://gogs.io/)是一款用Go语言实现的源代码管理系统，我们将用它来存储源代码。

部署Gogs非常简单

In [ ]:
!kubectl create -f gogs-rc.yaml

In [ ]:
!kubectl create -f gogs-svc.yaml

在部署完成后，我们访问 `http://$SERVER_IP:30300` 打开gogs系统，完成初始化，创建管理员帐号`root`，进入系统。

## 部署Drone

[Drone](https://github.com/drone/drone)是一款基于Docker的持续集成系统。我们将使用它将一个Git Repo打包成Docker Image，再推送到镜像仓库，完成部署。

部署Drone也是同样的流程

In [ ]:
!kubectl create -f drone-rc.yaml

In [ ]:
!kubectl create -f drone-svc.yaml

在部署完成后，我们访问 `http://$SERVER_IP:30380` 打开drone系统，使用gogs系统的管理员帐号登录，进入系统。

## 创建Git仓库

让我们在Gogs中创建一个名为`flaskr`的Repo。并在Drone中激活这个Repo的持续集成。

## 将flaskr源代码推送到Repo当中

首先将`ssh-key`添加到root用户之下

In [ ]:
!cat ~/.ssh/id_rsa.pub

再使用前面例子使用的flaskr源码，将它推送到repo

In [ ]:
!cp -r ../1-basic-app/flaskr .

In [ ]:
!git -C flaskr init

In [ ]:
!git -C flaskr add .

In [ ]:
!git -C flaskr config user.email "git@tryk8s.com"

In [ ]:
!git -C flaskr config user.name "TryK8S"

In [ ]:
!git -C flaskr commit -am 'Initial Commit'

In [ ]:
!git -C flaskr remote add origin git@gogs:root/flaskr.git

In [ ]:
!git -C flaskr push origin master

## 第一次集成

Drone对一个Repo进行持续集成的操作流程是定义在`.drone.yml`文件中，让我们将模板加入到Repo中，完成第一次集成

In [ ]:
!cp drone-first.yml flaskr/.drone.yml

In [ ]:
!git -C flaskr add .drone.yml

In [ ]:
!git -C flaskr commit -am 'First build'

In [ ]:
!git -C flaskr push origin master

当代码被推送到代码仓库后，Drone开始工作。让我们来看看Drone的工作是如何定义的。

In [ ]:
!cat drone-first.yml

`build`部分让Drone将当前Repo打包成镜像，推送到`registry:5000/flaskr`。`notify`部分通知Kubernetes创建一个名为`flaskr`的Deployment，完成部署。

Deployment的概念我们之前没接触过，简单来说是Kubernetes用来描述一次部署的资源。这里不做详细的解释。

让我们来看看部署是否成功

In [ ]:
!kubectl get pods

可以看到`flaskr`的Pod已经跑起来了，我们创建一个Service将它暴露到公网

In [ ]:
!kubectl create -f flaskr-svc.yaml

让我们用浏览器访问 `http://$SERVER_IP:30080`来确认服务已经可以正常启动了。

## 第二次集成

在第二次集成时，我们需要对`.drone.yml`文件做一处小的修改。

In [ ]:
!cat drone-second.yml

在第一次集成时，我们使用`POST`请求创建了一个新的Deployment，第二次我们使用`PUT`请求来修改已有的Deployment，完成同一套系统的自动更新。

接下来我们将这个变更发往源码仓库，激活第二次集成

In [ ]:
!cp drone-second.yml flaskr/.drone.yml

In [ ]:
!git -C flaskr commit -am 'Second build'

In [ ]:
!git -C flaskr push origin master

再确认新的版本已经成功被部署到系统当中

In [ ]:
!kubectl get pods

## 持续集成

之后的集成我们将在本地完成，让我们将这个Repo Clone到本地，做一些小的修改，再推送到代码仓库，检查我们做的更新是否成功的更新到线上系统当中。

## 总结

在使用Docker和Kubernetes这些集群系统的时候，一个普遍存在的困惑是该如何把本地代码在集群当中跑起来。在这个案例当中我们演示了使用持续集成系统将代码部署到集群当中的方法。当持续集成设置完成后，后续的系统更新只需要简单的`git push`就可以完成线上系统的更新，非常方便。

但是为了演示方便，我们简化了持续集成的流程。比如在一个版本的代码Build完成后，最好通过IM通知工程师，让工程师通过ChatOps下发部署指令，完成版本更新。

在这里我们主要的目的是把框架搭建起来，后续的改进还需要每一个团队根据自己的实际需要来进行调整。

## 资源释放

释放案例中创建的资源

In [ ]:
!kubectl delete deployment flaskr

In [ ]:
!kubectl delete svc flaskr

In [ ]:
!kubectl delete rc drone

In [ ]:
!kubectl delete svc drone

In [ ]:
!kubectl delete rc gogs

In [ ]:
!kubectl delete svc gogs